# Segmentation des datasets par catégorie de véhicule

## Objectif

Créer des datasets spécialisés par catégorie de véhicule/usager pour tester l'hypothèse suivante :  
**Des modèles entraînés sur des sous-populations homogènes pourraient mieux capturer les patterns de gravité spécifiques à chaque catégorie.**

## Catégories définies

| Catégorie | Usagers | Codes BAAC (catv) |
|-----------|---------|-------------------|
| **voiture** | VL, VU, voiturette | 3, 7, 10 |
| **vehicule_leger** | Motos, scooters, cyclomoteurs, quads | 2, 30-36, 41-43 |
| **velo_edp** | Vélos, trottinettes, EDP | 1, 50, 60, 80 |
| **pieton** | Piétons | catu=3 (pas de véhicule) |
| **poids_lourd** | Camions, bus, tracteurs | 13-17, 20-21, 37-38 |

## Hypothèse métier

Les taux de gravité varient fortement selon le type d'usager :
- Les **usagers vulnérables** (piétons, 2RM, vélos) ont des taux de blessures graves plus élevés
- Les **occupants de véhicules lourds** sont mieux protégés

Cette segmentation permet d'adapter les features et potentiellement d'améliorer les prédictions.

## 1. Imports et chargement

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# Lecture depuis Parquet (plus rapide)
parquet_path = "data/output/dataset_passager.parquet"

if os.path.exists(parquet_path):
    df = pd.read_parquet(parquet_path)
    print(f"Dataset PASSAGER chargé depuis Parquet: {df.shape}")
else:
    # Fallback vers ancien chemin
    df = pd.read_csv("data/dataset_passager.csv", sep=";", decimal=",", encoding="utf-8-sig")
    print(f"Dataset PASSAGER chargé depuis ancien chemin: {df.shape}")

print(f"Colonnes: {df.columns.tolist()}")

Dataset PASSAGER chargé depuis Parquet: (594406, 16)
Colonnes: ['catu', 'sexe', 'vma', 'agg', 'atm', 'catv', 'grav_binary', 'grav_ordered', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'sexe_conducteur', 'age_conducteur', 'a_equipement_adapte', 'age']


## 2. Définition des catégories de véhicules

Les codes `catv` proviennent de la nomenclature BAAC (Bulletins d'Analyse des Accidents Corporels).

**Logique de classification :**
1. Un **piéton** est identifié par `catu=3` (catégorie d'usager), pas par le véhicule
2. Les autres usagers sont classés selon le code `catv` de leur véhicule
3. Les véhicules non classés (codes rares ou "autre") sont exclus

In [2]:
# Catégories de véhicules (codes BAAC catv)
CATV_VOITURE = [3, 7, 10]  # Voiturette, VL, VU
CATV_LEGER = [2, 30, 31, 32, 33, 34, 35, 36, 41, 42, 43]  # Cyclomoteur, motos, scooters, quads
CATV_VELO = [1, 50, 60, 80]  # Vélo, EDP
CATV_PL = [13, 14, 15, 16, 17, 20, 21, 37, 38]  # PL, tracteurs, bus


def get_categorie(row):
    if row["catu"] == 3:
        return "pieton"
    elif row["catv"] in CATV_VOITURE:
        return "voiture"
    elif row["catv"] in CATV_LEGER:
        return "vehicule_leger"
    elif row["catv"] in CATV_VELO:
        return "velo_edp"
    elif row["catv"] in CATV_PL:
        return "poids_lourd"
    else:
        return "autre"


df["categorie"] = df.apply(get_categorie, axis=1)

print("Distribution par catégorie:")
print(df["categorie"].value_counts())
print(f"\nTotal: {len(df)}")

Distribution par catégorie:
categorie
voiture           387387
vehicule_leger    101403
pieton             46080
velo_edp           37073
poids_lourd        22463
Name: count, dtype: int64

Total: 594406


## 3. Distribution de la gravité par catégorie

**Objectif :** Vérifier si les taux de gravité diffèrent significativement entre catégories.

Si les taux sont très différents, des modèles spécialisés pourraient :
- Mieux apprendre les patterns propres à chaque population
- Éviter que les catégories majoritaires (voitures) dominent l'apprentissage

In [3]:
# Taux de gravité par catégorie
print("=== Taux de blessures graves (grav_binary=1) par catégorie ===\n")

stats = (
    df.groupby("categorie")
    .agg(nb_usagers=("grav_binary", "count"), nb_graves=("grav_binary", "sum"), taux_grave=("grav_binary", "mean"))
    .round(3)
)

stats["taux_grave_pct"] = (stats["taux_grave"] * 100).round(1)
print(stats.sort_values("taux_grave", ascending=False))

print("\n=== Taux équipement adapté par catégorie ===")
print(df.groupby("categorie")["a_equipement_adapte"].mean().round(3).sort_values(ascending=False))

=== Taux de blessures graves (grav_binary=1) par catégorie ===

                nb_usagers  nb_graves  taux_grave  taux_grave_pct
categorie                                                        
vehicule_leger      101403      34360       0.339            33.9
pieton               46080      15090       0.327            32.7
velo_edp             37073       9121       0.246            24.6
voiture             387387      45725       0.118            11.8
poids_lourd          22463       1305       0.058             5.8

=== Taux équipement adapté par catégorie ===
categorie
vehicule_leger    0.934
voiture           0.868
poids_lourd       0.757
velo_edp          0.389
pieton            0.005
Name: a_equipement_adapte, dtype: float64


## 4. Création des datasets par catégorie

Features adaptées selon la catégorie :
- **Piétons** : pas de `catv`, `sexe_conducteur`, `age_conducteur`
- **Vélo/EDP** : avec `catv`, mais sans `sexe_conducteur`, `age_conducteur` (le cycliste EST le conducteur)
- **Autres véhicules** : toutes les features

In [4]:
# Colonnes de base (communes à toutes les catégories)
# Note : on ne garde que les colonnes existantes dans le dataset
cols_base = [
    # Usager
    "age",
    "sexe",
    "catu",
    # Lieu
    "agg",
    "vma",
    # Météo
    "atm",
    # Temporel (features dérivées, pas heure brute)
    "est_nuit",
    "est_heure_pointe",
    "jour_semaine",
    "est_weekend",
    # Équipement
    "a_equipement_adapte",
    # Targets
    "grav_ordered",
    "grav_binary",
]

# Colonnes spécifiques aux véhicules motorisés
cols_vehicule_motorise = ["catv", "sexe_conducteur", "age_conducteur"]

# Colonnes pour vélo/EDP (catv mais pas conducteur car c'est l'usager lui-même)
cols_velo = ["catv"]


def make_dataset(df, categorie):
    """
    Crée un dataset pour une catégorie donnée.

    Les features sont adaptées selon le type d'usager :
    - Piétons : pas de catv ni infos conducteur (pas de véhicule)
    - Vélo/EDP : catv mais pas de conducteur (l'usager est le conducteur)
    - Autres : toutes les features
    """
    df_cat = df[df["categorie"] == categorie].copy()

    if categorie == "pieton":
        # Piétons : pas de véhicule
        cols = cols_base
    elif categorie == "velo_edp":
        # Vélo/EDP : catv mais pas de conducteur (c'est l'usager)
        cols = cols_base + cols_velo
    else:
        # Véhicules motorisés : toutes les colonnes
        cols = cols_base + cols_vehicule_motorise

    # Ne garder que les colonnes qui existent dans le dataset
    return df_cat[[c for c in cols if c in df_cat.columns]]


# Créer les datasets
datasets = {}
for cat in ["voiture", "vehicule_leger", "velo_edp", "pieton", "poids_lourd"]:
    datasets[cat] = make_dataset(df, cat)
    print(f"{cat}: {len(datasets[cat])} lignes, {len(datasets[cat].columns)} colonnes")

voiture: 387387 lignes, 16 colonnes
vehicule_leger: 101403 lignes, 16 colonnes
velo_edp: 37073 lignes, 14 colonnes
pieton: 46080 lignes, 13 colonnes
poids_lourd: 22463 lignes, 16 colonnes


## 5. Aperçu des datasets

In [5]:
print("=== Aperçu des datasets par catégorie ===\n")

for cat, df_cat in datasets.items():
    print(f"--- {cat.upper()} ---")
    print(f"Lignes: {len(df_cat)}")
    print(f"Colonnes: {df_cat.columns.tolist()}")
    print(f"Taux grave: {df_cat['grav_binary'].mean() * 100:.1f}%")
    print()

=== Aperçu des datasets par catégorie ===

--- VOITURE ---
Lignes: 387387
Colonnes: ['age', 'sexe', 'catu', 'agg', 'vma', 'atm', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'a_equipement_adapte', 'grav_ordered', 'grav_binary', 'catv', 'sexe_conducteur', 'age_conducteur']
Taux grave: 11.8%

--- VEHICULE_LEGER ---
Lignes: 101403
Colonnes: ['age', 'sexe', 'catu', 'agg', 'vma', 'atm', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'a_equipement_adapte', 'grav_ordered', 'grav_binary', 'catv', 'sexe_conducteur', 'age_conducteur']
Taux grave: 33.9%

--- VELO_EDP ---
Lignes: 37073
Colonnes: ['age', 'sexe', 'catu', 'agg', 'vma', 'atm', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'a_equipement_adapte', 'grav_ordered', 'grav_binary', 'catv']
Taux grave: 24.6%

--- PIETON ---
Lignes: 46080
Colonnes: ['age', 'sexe', 'catu', 'agg', 'vma', 'atm', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'a_equipement_adapte', 'grav_ordered', 

## 6. Export des datasets

In [6]:
# Export des datasets par catégorie (CSV + Parquet)
os.makedirs("data/output", exist_ok=True)

for cat, df_cat in datasets.items():
    # CSV
    csv_fname = f"data/output/dataset_passager_{cat}.csv"
    df_cat.to_csv(csv_fname, index=False, sep=";", decimal=",", encoding="utf-8-sig", quoting=csv.QUOTE_ALL)

    # Parquet
    parquet_fname = f"data/output/dataset_passager_{cat}.parquet"
    df_cat.to_parquet(parquet_fname, index=False)

    print(f"Exporté: {cat} ({len(df_cat)} lignes, {len(df_cat.columns)} colonnes)")
    print(f"  - CSV: {csv_fname}")
    print(f"  - Parquet: {parquet_fname}")

Exporté: voiture (387387 lignes, 16 colonnes)
  - CSV: data/output/dataset_passager_voiture.csv
  - Parquet: data/output/dataset_passager_voiture.parquet
Exporté: vehicule_leger (101403 lignes, 16 colonnes)
  - CSV: data/output/dataset_passager_vehicule_leger.csv
  - Parquet: data/output/dataset_passager_vehicule_leger.parquet
Exporté: velo_edp (37073 lignes, 14 colonnes)
  - CSV: data/output/dataset_passager_velo_edp.csv
  - Parquet: data/output/dataset_passager_velo_edp.parquet
Exporté: pieton (46080 lignes, 13 colonnes)
  - CSV: data/output/dataset_passager_pieton.csv
  - Parquet: data/output/dataset_passager_pieton.parquet
Exporté: poids_lourd (22463 lignes, 16 colonnes)
  - CSV: data/output/dataset_passager_poids_lourd.csv
  - Parquet: data/output/dataset_passager_poids_lourd.parquet


## Résumé

In [7]:
print("=== DATASETS CRÉÉS ===\n")
print(f"{'Dataset':<30} {'Lignes':>10} {'Colonnes':>10} {'Taux grave':>12}")
print("-" * 65)

# Global
print(f"{'dataset_passager.csv':<30} {len(df):>10} {len(df.columns) - 1:>10} {df['grav_binary'].mean() * 100:>11.1f}%")

# Par catégorie
for cat, df_cat in datasets.items():
    name = f"dataset_passager_{cat}.csv"
    print(f"{name:<30} {len(df_cat):>10} {len(df_cat.columns):>10} {df_cat['grav_binary'].mean() * 100:>11.1f}%")

=== DATASETS CRÉÉS ===

Dataset                            Lignes   Colonnes   Taux grave
-----------------------------------------------------------------
dataset_passager.csv               594406         16        17.8%
dataset_passager_voiture.csv       387387         16        11.8%
dataset_passager_vehicule_leger.csv     101403         16        33.9%
dataset_passager_velo_edp.csv       37073         14        24.6%
dataset_passager_pieton.csv         46080         13        32.7%
dataset_passager_poids_lourd.csv      22463         16         5.8%


## Usage

### Modèle global
```python
df = pd.read_parquet('data/output/dataset_passager.parquet')
```

### Modèle spécialisé par catégorie
```python
df_voiture = pd.read_csv('data/output/dataset_passager_voiture.csv', sep=';', decimal=',')
df_vehicule_leger = pd.read_csv('data/output/dataset_passager_vehicule_leger.csv', sep=';', decimal=',')
# etc.
```

### Intérêt de la segmentation

- **Taux de gravité différents** : les véhicules légers et piétons ont généralement des taux plus élevés
- **Features différentes** : les piétons n'ont pas de véhicule/conducteur
- **Modèles spécialisés** : peuvent mieux capturer les patterns de chaque catégorie